<a href="https://colab.research.google.com/github/rdezwart/Computer-Vision-Project/blob/dev/FoodClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food Classification Using YOLO

## Required Libraries


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
import seaborn as sns
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/78.2 GB disk)


In [4]:
from ultralytics import YOLO

## Training YOLO

In [5]:
model = YOLO('yolov8s-cls.pt')

100%|██████████| 12.2M/12.2M [00:00<00:00, 344MB/s]


Ignore this config file, turns out it's not needed for classification.

In [35]:
%%writefile config.yaml
path: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db
train: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/train
test: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/test
val: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/eval

# Classes
nc: 11 # replace based on your dataset's number of classes

# Class names
# replace all class names with your own classes' names
names:
  0: bread
  1: dairy
  2: dessert
  3: egg
  4: fried
  5: meat
  6: noodles
  7: produce
  8: rice
  9: seafood
  10: soup

Overwriting config.yaml


In [7]:
model.train(data="/content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db",epochs=1,batch=16)

Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db, epochs=1, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fa

train: Scanning /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/train... 9866 images, 0 corrupt: 100%|██████████| 9866/9866 [00:00<?, ?it/s]
val: Scanning /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/val... 3347 images, 0 corrupt: 100%|██████████| 3347/3347 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 224 train, 224 val
Using 2 dataloader workers
Logging results to runs/classify/train2
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size


        1/1     0.512G      2.464         16        224:   1%|          | 7/617 [00:02<02:53,  3.52it/s]

        1/1     0.512G      2.442         16        224:   2%|▏         | 11/617 [00:04<04:16,  2.37it/s]
100%|██████████| 755k/755k [00:00<00:00, 95.3MB/s]
               classes   top1_acc   top5_acc: 100%|██████████| 105/105 [16:10<00:00,  9.25s/it]


                   all       0.84      0.992

1 epochs completed in 1.308 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 10.3MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 10.3MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-cls summary (fused): 73 layers, 5089291 parameters, 0 gradients, 12.5 GFLOPs
train: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/train... found 9866 images in 11 classes ✅ 
val: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/val... found 3347 images in 11 classes ✅ 
test: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/test... found 3430 images in 11 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 105/105 [00:42<00:00,  2.50it/s]


                   all       0.84      0.992
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
Results saved to runs/classify/train2


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fbf48421b10>
curves: []
curves_results: []
fitness: 0.9160442650318146
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8398566246032715, 'metrics/accuracy_top5': 0.9922319054603577, 'fitness': 0.9160442650318146}
save_dir: PosixPath('runs/classify/train2')
speed: {'preprocess': 0.0909292274067072, 'inference': 0.49147023205832646, 'loss': 0.0007161817730167641, 'postprocess': 0.0005635995810730691}
task: 'classify'
top1: 0.8398566246032715
top5: 0.9922319054603577

## Evaluation Metrics

In [16]:
metrics = model.val()  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
train: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/train... found 9866 images in 11 classes ✅ 
val: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/val... found 3347 images in 11 classes ✅ 
test: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/test... found 3430 images in 11 classes ✅ 


val: Scanning /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/val... 3347 images, 0 corrupt: 100%|██████████| 3347/3347 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 210/210 [00:43<00:00,  4.81it/s]


                   all       0.84      0.992
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train25


## Prediction

In [ ]:
results = model()